## SV Calling 
-----

In [1]:
##local
projectD=/home/sarah/WGS3          ##path to project folder 
cd $projectD
mkdir testSV                       ##name of SV folder
svFolder=testSV                    ##variable for name

In [ ]:
svFolder= path/to/sv/foldertestSV            
cd $svFolder            
chmod u+x runScript.sh
refs=$projectD/refs  ##replace refs with reference folder name 
bams=$projectD/bams  ##replace bams with bam folder name
reference=$refs/refSeqName  ##replace refSeqName with name of reference file inside of refs folder 
insertSize = 250            ##input estimated insert size based on sequence data 

./runScript.sh -d $mydata -b $bams -r $reference -i 250

In [ ]:
##converting delly from bcf to vcf 
cd $projectD/$svFolder/delly
for file in *.bcf ;
do
out=${file%.bcf}.vcf
bcftools view -Ov -o $out $file
done

##annotating gridss output
chmod u+x $projectD/$svFolder/structurial_annotation.R 
cd $projectD/$svFolder/gridss
for file in *.vcf ;
do
outVcf=${file%.vcf}.ann.vcf
outBed=${file%.vcf}.bed
Rscript $projectD/$svFolder/structurial_annotation.R $file $outVcf $outBed
done

#annotating lumpy output
cd $projectD/$svFolder/lumpy
for file in *.vcf ;
do
outVcf=${file%.vcf}.ann.vcf
outBed=${file%.vcf}.bed
Rscript $projectD/$svFolder/structurial_annotation.R $file $outVcf $outBed
done

In [ ]:
#run reformat code for each sample
for b in $projectD/$svFolder/stats/*.stats
do
tmp=${b##*/}
sample=${tmp%.sorted.stats}
python3 $projectD/$svFolder/reformat_merge.py $projectD/$svFolder $sample
done

In [ ]:
numSamples=20                   ## replace 20 with number of samples 
python3 $projectD/$svFolder/overlapSamples.py $projectD/$svFolder/mergeSV $numSamples

In [ ]:
cd $svFolder
mkdir -p svImages 
bedtools igv -slop 10 -path svImages -i $projectD/$svFolder/mergeSV/noShareSV.bed > $projectD/$svFolder/svImages/picScript.sh

In [ ]:
### TODO generate images into IGV make session files 

In [ ]:
cd $svFolder/svImages
for photo in *.png ; do convert $photo -rotate -90 $photo ; done